In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [2]:
dataframe_machine = pd.read_csv("machine.data", header = None)
dataframe_housing = pd.read_csv("housing.data", header = None, sep=r"\s+")
dataframe_communities = pd.read_csv("communities.data", header = None)
dataframe_wpbc = pd.read_csv("wpbc.data", header = None)

In [3]:
"""Computer Hardware Data Set"""
dataframe_machine = dataframe_machine.drop([0, 1, 9], axis = 1)
dataframe_machine_X = dataframe_machine.values[:, :-1]
dataframe_machine_Y = dataframe_machine.values[:, -1]

In [4]:
"""Boston Housing"""
dataframe_housing_X = dataframe_housing.values[:, :-1]
dataframe_housing_Y = dataframe_housing.values[:, -1]


In [5]:
"""Wisconsing Breast Cancer"""
dataframe_wpbc=dataframe_wpbc.drop([1], axis=1)
dataframe_wpbc_X = dataframe_wpbc.values[:, :-1]
dataframe_wpbc_Y = dataframe_wpbc.values[:, -1]

In [6]:
"""Communities and Crime"""
dataframe_communities = dataframe_communities.drop([0,1,2,3,4], axis=1)
missing_index = dataframe_communities.columns[(dataframe_communities == '?').any()].tolist()
dataframe_communities = dataframe_communities.drop(missing_index, axis=1)

dataframe_communities_X = dataframe_communities.values[:, :-1]
dataframe_communities_Y = dataframe_communities.values[:, -1]

C:\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [7]:
data = []

In [8]:
def errors(search, x_train, x_test, y_train, y_test):
    errors_list = []
    #Train 
    errors_list.append(cross_val_score(search, x_train, y_train, cv=3, scoring='neg_mean_absolute_error') * -1)
    errors_list.append(cross_val_score(search, x_train, y_train, cv=3, scoring='neg_mean_squared_error') * -1)
    errors_list.append(cross_val_score(search, x_train, y_train, cv=3, scoring='neg_median_absolute_error') * -1)
    #Test
    errors_list.append(cross_val_score(search, x_test, y_test, cv=3, scoring='neg_mean_absolute_error') * -1)
    errors_list.append(cross_val_score(search, x_test, y_test, cv=3, scoring='neg_mean_squared_error') * -1)
    errors_list.append(cross_val_score(search, x_test, y_test, cv=3, scoring='neg_median_absolute_error') * -1)
    return errors_list

In [9]:
def Random_Search(parameter, model, x_train, x_test, y_train, y_test):
    randomized_search = RandomizedSearchCV(estimator = model, param_distributions = parameter, 
                                scoring='neg_mean_squared_error', cv=3, return_train_score=True, iid=False)
    randomized_search.fit(x_train, y_train)
    return ['Random Search'] + errors(randomized_search, x_train, x_test, y_train, y_test)

In [10]:
def Grid_Search(parameter, model, x_train, x_test, y_train, y_test):
    grid_search = GridSearchCV(estimator = model, param_grid = parameter, 
                          scoring='neg_mean_squared_error', cv=3, return_train_score=True, iid=False)
    grid_search.fit(x_train, y_train)
    return ['Grid Search'] + errors(grid_search, x_train, x_test, y_train, y_test)

In [11]:
"""K cel mai apropiat vecin este un algoritm simplu care stochează toate cazurile disponibile și prezice ținta numerică 
pe baza unei măsuri de similaritate (de exemplu, funcțiile de distanță). KNN a fost utilizată în estimarea statistică și 
recunoașterea modelelor la începutul anilor 1970 ca o tehnică non-parametrică. O simplă implementare a regresiei KNN este 
de a calcula media țintei numerice a celor mai apropiați vecini K. O altă abordare folosește o medie ponderată inversă a 
celor mai apropiați vecini K. Regresia KNN utilizează aceleași funcții de distanță ca și clasificarea KNN. Formulele obisnuite
pentru masurarea distantei sunt valabile numai pentru variabilele continue. În cazul variabilelor categorice, trebuie 
să utilizați formula pentru distanța Hamming, care reprezintă o măsură a numărului de cazuri în care simbolurile corespunzătoare
sunt diferite în două șiruri de aceeași lungime.Alegerea valorii optime pentru K se face cel mai bine prin prima inspecție a 
datelor. În general, o valoare K mare este mai precisă, deoarece reduce zgomotul total; cu toate acestea, compromisul este că 
limitele distincte din spațiul caracteristicilor sunt neclare. Validarea încrucișată este o altă modalitate de determinare 
retrospectivă a unei valori K bune prin utilizarea unui set de date independent pentru a valida valoarea K. K optimal pentru 
majoritatea seturilor de date este de 10 sau mai mult. Asta produce rezultate mult mai bune decât 1-NN.
KNN poate fi folosit atât pentru probleme de predicție de clasificare, cât și pentru regresie. Cu toate acestea, este utilizat
mai mult în problemele de clasificare din industrie. Pentru a evalua orice tehnică, analizăm în general trei aspecte importante:
Ușor de interpretat ieșirea, Timpul de calcul, Puterea predictivă"""
""""https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/"""""
"""https://www.saedsayad.com/k_nearest_neighbors_reg.htm"""


def KNN_Regressor(x_train, y_train, x_test, y_test):
    parameter = {'n_neighbors': list(range(1,12)), 'p': [1, 2, 3, 4.7]} 
    model = KNeighborsRegressor()
    data.append(['KNN'] + Grid_Search(parameter, model, x_train, x_test, y_train, y_test))
    data.append(['KNN'] + Random_Search(parameter, model, x_train, x_test, y_train, y_test))

![asdfg](knn.jpg)

In [12]:
""" SVM este o clasă foarte specifică a algoritmului, caracterizată prin utilizarea de kerneluri, absența minimelor locale, 
raritatea soluției și controlul capacității obținute prin acționarea pe margine sau pe numărul de vectori de sprijin etc.A fost 
inventat de Vladimir Vapnik și colegii săi și au fost introduși pentru prima oară la conferința cu lucrarea "Teoria învățării 
computaționale (COLT) 1992". Toate aceste caracteristici frumoase totuși au fost deja prezente în procesul de învățare mecanică 
începând cu anii 1960: utilizarea hiper-avioanelor de margini mari a kernelurilor, interpretarea geometrică a kernelurilor 
ca produse interioare într-un spațiu caracteristic. Tehnici similare de optimizare au fost folosite în recunoașterea modelului 
și tehnicile de spargare au fost discutate pe scară largă. Utilizarea variabilelor slabe pentru depășirea zgomotului în date 
și a separabilității a fost introdusă, de asemenea, în anii 1960. Cu toate acestea, până în anul 1992 toate aceste caracteristici
au fost puse împreună pentru a forma clasificatorul de marjă maximă, mașina de suport de bază, și nu până în 1995, că a fost 
introdusă versiunea de marjă moale.
SVM poate fi aplicată nu numai problemelor de clasificare, ci și în cazul regresiei. Totuși, acesta conține toate 
caracteristicile principale care caracterizează algoritmul de marjă maximă: o funcție neliniară este înclinată prin 
cartografiere liniară de învățare a mașinilor în spațiul caracteristic de dimensiuni mari induse de kernel. Capacitatea 
sistemului este controlată de parametrii care nu depind de dimensionalitatea spațiului caracteristic. În același mod ca și 
abordarea de clasificare există motivația de a căuta și de a optimiza limitele de generalizare date pentru regresie. Ei s-au 
bazat pe definirea funcției de pierdere care ignoră erorile care se află în anumite distanțe ale valorii reale. Acest tip de 
funcție este adesea numit - funcția de pierdere intensivă - epsilon. Figura de mai jos prezintă un exemplu de funcție de 
regresie liniară unidimensională cu banda intensivă - epsilon. Variabilele măsoară costul erorilor pe punctele de antrenament. 
Acestea sunt zero pentru toate punctele care se află în interiorul benzii. """
"""http://kernelsvm.tripod.com/"""
"""https://www.saedsayad.com/support_vector_machine_reg.htm"""
def SVM_Regressor(x_train, y_train, x_test, y_test):
    parameter = {'C': list([0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 2.0, 3.0, 4.0, 5.0, 10.0])}
    model = SVR()
    data.append(['SVC'] + Grid_Search(parameter, model, x_train, x_test, y_train, y_test))
    data.append(['SVC'] + Random_Search(parameter, model, x_train, x_test, y_train, y_test))

![SVM regressor](svm.png)

In [13]:
"""Random forest sau pădurile decizionale aleatoare reprezinta o metodă de învățare a ansamblului pentru clasificare, regresie 
și alte sarcini care operează prin construirea unei multitudini de arbori de decizie la timpul de instruire și scoaterea clasei 
care este modul de clasificare (clasificare) sau predicție medie (regresie) a copacilor individuali. Pădurile decizionale 
aleatorii corectează obiceiul copiilor decizionali de a suprasolicita la setul lor de antrenament. Primul algoritm pentru 
pădurile decizionale aleatorii a fost creat de Tin Kam Ho utilizând metoda subspațiului aleatoriu, care, în formula lui Ho, 
este o modalitate de a pune în practică abordarea de clasificare propusă de Eugene Kleinberg. O extensie a algoritmului a fost 
dezvoltată de Leo Breiman și Adele Cutler, care au înregistrat "Forest Random" ca marcă comercială (din 2019, deținută de 
Minitab, Inc.) Extensia combină ideea "bagging" a lui Breiman și selecția aleatorie a caracteristicilor, introduse mai întâi 
de Ho și mai târziu independent de Amit și Geman pentru a construi o colecție de arbori de decizie cu variație controlată. 
Decision trees reprezintă o metodă populară pentru diferitele activități de învățare a mașinilor. Arborele de învățare "vine 
cel mai aproape de a îndeplini cerințele pentru a servi ca o procedură de tip" off-the-shelf "pentru miniere de date", spune 
Hastie și colab., "Deoarece este invariantă sub scalare și diverse alte transformări ale valorilor caracteristice, la includerea
caracteristicilor irelevante și produce modele inspectabile, însă acestea sunt rareori exacte". 
"O pădure aleatoare este o tehnică de ansamblu capabilă să efectueze atât sarcini de regresie, cât și sarcini de clasificare, 
folosind mai mulți copaci de decizie și o tehnică numită Bootstrap Aggregation, cunoscută în mod obișnuit ca "bagging". 
Ce poți să-ți pui bagajul? Bagging-ul, în metoda Random Forest, implică instruirea fiecărui arbore al deciziilor pe un eșantion 
de date diferit, unde prelevarea de probe se face cu înlocuirea. Ideea de bază din spatele acestei situații este combinarea 
mai multor arbori de decizie în determinarea producției finale, în loc să se bazeze pe arborii decizionali individuali. Dacă 
doriți să citiți mai multe despre Random Forest, am inclus câteva link-uri de referință care oferă explicații detaliate pe 
această temă."""
"""https://medium.com/datadriveninvestor/random-forest-regression-9871bc9a25eb"""
"""https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd"""

def Random_Forest_Regressor(x_train, y_train, x_test, y_test):
    parameter = {'n_estimators': list(range(1,13))}
    model = RandomForestRegressor()
    data.append(['Forest'] + Grid_Search(parameter, model, x_train, x_test, y_train, y_test))
    data.append(['Forest'] + Random_Search(parameter, model, x_train, x_test, y_train, y_test))

![Random_Forest](random-forest.png)

In [14]:
"""Decizia Arbore este un instrument de luare a deciziilor care utilizează o structură arborescentă ca un diagramă sau este un 
model de decizii și toate rezultatele posibile ale acestora, inclusiv rezultatele, costurile de intrare și utilitatea. 
Algoritmul algoritmului de decizie se încadrează în categoria algoritmilor de învățare supravegheați. Funcționează atât pentru 
variabilele de ieșire atât continue, cât și categorice. Ramurile / marginile reprezintă rezultatul nodului și nodurile au:
condiții [noduri de decizie], rezultat [noduri finale], ramurile / marginile reprezintă adevărul / falsitatea afirmației și ia 
o decizie bazată pe cea din exemplul de mai jos care arată un arbore de decizie care evaluează cel mai mic dintre cele trei 
numere. Succesiunea regresiei arborelui observă caracteristicile unui obiect și pregătește un model din structura unui copac 
pentru a prezice date în viitor pentru a produce o ieșire continuă semnificativă. Ieșirea continuă înseamnă că rezultatul / 
rezultatul nu este discret, adică nu este reprezentat doar de un set discret, cunoscut de numere sau valori.
În domeniul informaticii, învățarea arborelui decizional folosește un arbore de decizie (ca model predictiv) pentru a trece de 
la observațiile despre un element (reprezentat în ramuri) la concluziile privind valoarea țintă a elementului (reprezentată în 
frunze). Este una dintre abordările de modelare predictivă utilizate în statistici, în domeniul minelor de date și al învățării
în mașină. Modelele de copaci în care variabila țintă poate lua un set discret de valori se numesc arbori de clasificare; în 
aceste structuri de copaci, frunzele reprezintă etichete de clasă, iar ramificațiile reprezintă conjuncții ale caracteristicilor
care conduc la acele etichete de clasă. Arborii de decizie în care variabila țintă pot lua valori continue (de obicei numere 
reale) se numesc arbori de regresie. În analiza deciziei, un arbore de decizie poate fi folosit pentru a reprezenta în mod 
vizual și explicit deciziile și luarea deciziilor. În exploatarea datelor, un arbore de decizie descrie datele (dar arborele de 
clasificare rezultat poate fi o contribuție pentru luarea deciziilor). Această pagină se ocupă cu arborii de decizie în 
domeniul exploatării datelor."""
"""https://www.saedsayad.com/decision_tree_reg.htm"""
"""https://en.wikipedia.org/wiki/Decision_tree_learning"""

def Decision_Tree_Regressor(x_train, y_train, x_test, y_test):
    parameter = {'random_state': list([1, 50, 100, 200, 300, 333, 500, 700, 1000, 2000, 3000, 10000])}
    model = DecisionTreeRegressor()
    data.append(['Tree'] + Grid_Search(parameter, model, x_train, x_test, y_train, y_test))
    data.append(['Tree'] + Random_Search(parameter, model, x_train, x_test, y_train, y_test))

![Decision_Tree](decision-tree.png)

In [15]:
"""În teoria și statisticile de probabilitate, un proces Gaussian este un proces stochastic (o colecție de variabile aleatoare 
indexate de timp sau de spațiu), astfel încât fiecare colecție finită a acestor variabile aleatoare are o distribuție normală 
multivariată, adică fiecare combinație liniară finită a acestora este în mod normal distribuite. Distribuția unui proces 
Gaussian este distribuția comună a tuturor acelor variabile aleatoare (infinit de multe) și, ca atare, este o distribuție 
asupra funcțiilor cu un domeniu continuu, de ex. timp sau spațiu. Un algoritm de învățare a mașinilor care implică un proces 
Gaussian utilizează învățarea leneș și o măsură a asemănării dintre puncte (funcția kernelului) pentru a prezice valoarea 
pentru un punct nevăzut din datele de antrenament. Predicția nu este doar o estimare pentru acel punct, ci și o informație de 
incertitudine - este o distribuție Gaussiană unidimensională (care este distribuția marginală la acel punct). Pentru unele 
funcții de kernel, algebra matricială poate fi folosită pentru a calcula predicțiile folosind tehnica kriging. Când se 
utilizează un kernel parametrizat, software-ul de optimizare este de obicei folosit pentru a se potrivi unui model de proces 
Gaussian. Conceptul de procese Gauss este numit după Carl Friedrich Gauss pentru că se bazează pe noțiunea de distribuție 
Gaussiană (distribuție normală). Procesele Gaussian pot fi văzute ca o generalizare infinită-dimensională a distribuțiilor 
normale multivariate. Procesele gaussiene sunt utile în modelarea statistică, beneficiind de proprietăți moștenite din 
distribuția normală. Un proces Gaussian poate fi folosit ca distribuție de probabilitate anterioară față de inferența Bayesiană
Având în vedere orice set de puncte N în domeniul dorit al funcțiilor dumneavoastră, luați un multivariat Gaussian a cărui 
parametru de matrice covarianță este matricea Gram a punctelor dvs. N cu un kernel dorit și o mostră din acel Gaussian. 
Inferența valorilor continue cu un proces Gaussian este cunoscută sub denumirea de regresie a procesului Gaussian, sau kriging; 
Regresia procesului Gaussian pentru variabilele țintă multiple este cunoscută sub denumirea de cocriging.Procesele Gauss sunt 
astfel utile ca un instrument puternic de interpolare neliniară multivariabilă. Regresia procesului Gauss poate fi extinsă în 
continuare pentru a aborda sarcinile de învățare atât în cadrele de învățare supravegheate (de exemplu, în clasificarea 
probabilistică), cât și în cele nesupravegheate"""
"""www.gaussianprocess.org/gpml/chapters/RW2.pdf"""
"""https://en.wikipedia.org/wiki/Gaussian_process"""


def Gaussian_Process_Regressor(x_train, y_train, x_test, y_test):
    parameter = {'alpha': list([0.0000000000001, 0.000000000001, 0.00000000001, 0.0000000001, 0.000000001, 0.00000001, 0.0000001, 
                                        0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0])}
    model = GaussianProcessRegressor()
    data.append(['GaussianProcessRegressor'] + Grid_Search(parameter, model, x_train, x_test, y_train, y_test))
    data.append(['GaussianProcessRegressor'] + Random_Search(parameter, model, x_train, x_test, y_train, y_test))

![Gaussian_Process](gaussian.png)

In [16]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

In [17]:
def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: red' if v else '' for v in is_min]

In [21]:
def Run(x, y, name):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)
    
    KNN_Regressor(x_train, y_train, x_test, y_test)
    Random_Forest_Regressor(x_train, y_train, x_test, y_test)
    Decision_Tree_Regressor(x_train, y_train, x_test, y_test)
    Gaussian_Process_Regressor(x_train, y_train, x_test, y_test)
    data_frame = pd.DataFrame(data, columns=['Model', 'Search', 'train_mean_absolute_error', 'train_mean_squared_error',
                                        'train_median_absolute_error', 'test_mean_absolute_error', 'test_mean_squared_error', 
                                         'test_median_absolute_error'])
    #data_frame = data_frame.style.apply(highlight_max, subset=['train_mean_absolute_error', 'train_mean_squared_error',
                                        #'train_median_absolute_error', 'test_mean_absolute_error', 'test_mean_squared_error', 
                                        #'test_median_absolute_error']).apply(highlight_min, subset=['train_mean_absolute_error',
                                        #'train_mean_squared_error', 'train_median_absolute_error', 'test_mean_absolute_error',
                                        #'test_mean_squared_error', 'test_median_absolute_error'])
    return data_frame

In [22]:
Run(dataframe_machine_X, dataframe_machine_Y, "machine").to_html('Machine.html')

In [23]:
Run(dataframe_housing_X, dataframe_housing_Y, "housing").to_html('Housing.html')

In [ ]:
Run(dataframe_wpbc_X, dataframe_wpbc_Y, "wpbc").to_html('Wpbc.html')

In [ ]:
Run(dataframe_communities_X, dataframe_communities_Y, "communities").to_html("Comunities.html")